Modelo bimodal



In [ ]:
!pip install -U transformers[torch]
!pip install verovio
!pip install tiktoken
!pip install gradio
!pip install ultralytics
!pip install python-dotenv
!pip install inference_sdk

In [ ]:
from google.colab import drive
from transformers import AutoModel, AutoTokenizer
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display
from dotenv import load_dotenv
import gradio as gr
from inference_sdk import InferenceHTTPClient

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('ucaslcl/GOT-OCR2_0', trust_remote_code=True)
model = AutoModel.from_pretrained('ucaslcl/GOT-OCR2_0', trust_remote_code=True, low_cpu_mem_usage=True, device_map='cuda', use_safetensors=True, pad_token_id=tokenizer.eos_token_id)
model = model.eval().cuda()


# input your test image
image_file = '/content/drive/Shareddrives/Final/dataset/PS5/Escaneado_ 20241106-1911-01.jpg'

# plain texts OCR
res = model.chat(tokenizer, image_file, ocr_type='ocr')

print(res)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


BP S S CALL DUTY BLACK OPS 18 SE NEC ES IT A CONE XION A INTERNET www. pegi. info REO U IERO EDE SCAR GA DE CONTE NI OO Com ple tamente en castellano


In [ ]:
display.clear_output()

In [ ]:
# Cargar variables de entorno desde el archivo .env
load_dotenv('.env')

# Obtener la clave de API desde la variable de entorno
API_KEY = os.getenv("ROBOFLOW_API_KEY")

In [ ]:
# initialize the client
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key=API_KEY
)

# infer on a local image
result = CLIENT.infer(image_file, model_id="clasificacion-videojuegos-2.0/2")

print(result)




{'inference_id': '59a095ba-df9b-47fc-9b2f-f5ea8c245c3b', 'time': 0.12450575399998343, 'image': {'width': 2415, 'height': 3054}, 'predictions': [{'x': 241.0, 'y': 2734.5, 'width': 340.0, 'height': 413.0, 'confidence': 0.9663149118423462, 'class': 'PEGI 18', 'class_id': 4, 'detection_id': '62fbd172-bb0f-4c01-8e10-a00db620bae3'}, {'x': 606.5, 'y': 217.0, 'width': 1145.0, 'height': 318.0, 'confidence': 0.9557659029960632, 'class': 'PS5', 'class_id': 8, 'detection_id': '9c237fb7-94dc-4eff-b2c5-7285d39470e2'}]}


In [ ]:
def clasificacion(image):

  # YOLO
  CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="zuGPdxGRyCVN3EtRgecJ"
)

  yolo_result = CLIENT.infer(image, model_id="clasificacion-videojuegos-2.0/2")
  predicciones = yolo_result["predictions"]
  clase1 = predicciones[0]["class"]
  clase2 = predicciones[1]["class"]

  # OCR
  tokenizer = AutoTokenizer.from_pretrained('ucaslcl/GOT-OCR2_0', trust_remote_code=True)
  model = AutoModel.from_pretrained('ucaslcl/GOT-OCR2_0', trust_remote_code=True, low_cpu_mem_usage=True, device_map='cuda', use_safetensors=True, pad_token_id=tokenizer.eos_token_id)
  model = model.eval().cuda()

  ocr_result = model.chat(tokenizer, image, ocr_type='ocr')

  return clase1, clase2, ocr_result




Interfaz Gradio

In [ ]:
gr.Interface(
    fn=clasificacion,
    inputs=[
        gr.Image(type="filepath", label="Sube una imagen JPG"),  # Componente para subir imágenes
    ],
    outputs=[
        gr.Textbox(label="Clase 1"),  # Caja de texto para la primera clase
        gr.Textbox(label="Clase 2"),  # Caja de texto para la segunda clase
        gr.Textbox(label="Resultado OCR"),  # Caja de texto para el resultado del OCR
    ],
    title="Clasificación y OCR con YOLO y Roboflow",
    description="Sube una imagen JPG para obtener las clases detectadas por YOLO y el texto reconocido por OCR."
).launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0cfe7d9904de0b0813.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
